In [16]:
import pandas as pd
import numpy as np
from pandas import DataFrame
import sqlite3

import warnings
warnings.filterwarnings("ignore")



def prepare(df_drug, feature_list,mechanism,action,drugA,drugB):

    d_label = {}
    d_feature = {}

    # Transfrom the interaction event to number
    d_event=[]
    for i in range(len(mechanism)):
        d_event.append(mechanism[i]+" "+action[i])

    count={}
    # 打标签
    for i in d_event:
        if i in count:
            count[i]+=1
        else:
            count[i]=1
    
    # interaction的种类数
    event_num=len(count)
    
    # list1按照interaction的实体数量顺序排列
    # list1 like [('A B', 100), ('C D', 50), ('E F', 30), ('G H', 20)]
    list1 = sorted(count.items(), key=lambda x: x[1],reverse=True)
    each_event_num=[]
    
    # d_label like {'A B': 0, 'C D': 1, 'E F': 2, 'G H': 3}，这步等于是把元组列表转换为字典，同时得到标签
    # 同时得到每个interaction的数量
    for i in range(len(list1)):
        d_label[list1[i][0]]=i
        each_event_num.append(list1[i][1])

    vector = np.zeros((len(np.array(df_drug['name']).tolist()), 0), dtype=float)  #vector=[]
    
    for i in feature_list:
        #vector = np.hstack((vector, feature_vector(i, df_drug, vector_size)))#1258*1258
        tempvec=feature_vector(i, df_drug)
        
        # 把特征向量沿着列的方向拼接起来
        vector = np.hstack((vector,tempvec))
        
    # Transfrom the drug ID to feature vector
    for i in range(len(np.array(df_drug['name']).tolist())):
        d_feature[np.array(df_drug['name']).tolist()[i]] = vector[i]

    print("d_feature shape is:", d_feature['Glucosamine'].shape)
    # Use the dictionary to obtain feature vector and label
    new_feature = []
    new_label = []

    for i in range(len(d_event)):
        temp=np.hstack((d_feature[drugA[i]],d_feature[drugB[i]]))
        new_feature.append(temp)
        new_label.append(d_label[d_event[i]])

    new_feature = np.array(new_feature) #323539*....
    new_label = np.array(new_label)  #323539

    print("new_feature shape is:", new_feature.shape)
    print("new_label shape is:", new_label.shape)
    # 返回的是每个三元组的特征向量，标签，interaction种类的数量，每类interaction的出现次数
    return new_feature, new_label,event_num,each_event_num


# In[104]:


def feature_vector(feature_name, df):
    def Jaccard(matrix):
        matrix = np.mat(matrix)

        numerator = matrix * matrix.T

        denominator = np.ones(np.shape(matrix)) * matrix.T + matrix * np.ones(np.shape(matrix.T)) - matrix * matrix.T

        return numerator / denominator
    
    all_feature = []
    drug_list = np.array(df[feature_name]).tolist()
    
    # print(drug_list)
    
    # Features for each drug, for example, when feature_name is target, drug_list=["P30556|P05412","P28223|P46098|……"]
    for i in drug_list:
        for each_feature in i.split('|'):
            if each_feature not in all_feature:
                all_feature.append(each_feature)  # obtain all the features
    feature_matrix = np.zeros((len(drug_list), len(all_feature)), dtype=float)
    df_feature = DataFrame(feature_matrix, columns=all_feature)  # Consrtuct feature matrices with key of dataframe
    
    # 做特征矩阵，每一行代表一个药物，每一列代表一个特征，如果药物有这个特征，那么这个位置就是1；0，1矩阵
    for i in range(len(drug_list)):
        for each_feature in df[feature_name].iloc[i].split('|'):
            df_feature[each_feature].iloc[i] = 1
    
    df_feature = np.array(df_feature)
    
    sim_matrix = np.array(Jaccard(df_feature))
    
    print(feature_name+" len is:"+ str(len(sim_matrix[0])))

    return sim_matrix


conn = sqlite3.connect("/data/dingli/mydata/MDDI-SCL/datasets/MDF-SA-DDI/event.db")

# 多层次知识图谱数据集
df_drug = pd.read_sql('select * from drug;', conn)
extraction = pd.read_sql('select * from extraction;', conn)
feature_list = ["smile","target","enzyme","pathway"]

mechanism = extraction['mechanism']
action = extraction['action']
drugA = extraction['drugA']
drugB = extraction['drugB']

new_feature, new_label, event_num,each_event_num=prepare(df_drug,feature_list,mechanism,action,drugA,drugB)

smile len is:572
target len is:572
enzyme len is:572
pathway len is:572
d_feature shape is: (2288,)
new_feature shape is: (37264, 4576)
new_label shape is: (37264,)
